In [2]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [3]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
from chat_template_utils import get_json_schema
from llama_cpp import Llama
from utils import *
from peft import *
import datasets
import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0it [00:00, ?it/s]

In [4]:
train_dataset = pd.read_excel('train_forms.xlsx')[['input_text','labels']]
train_dataset.drop_duplicates(['input_text']).reset_index(drop=True)
c = train_dataset['labels']=='other'
train_dataset.loc[c,'labels']=pd.NA
c = train_dataset['labels']=='birthdate'
train_dataset.loc[c,'labels']='birth'

train_dataset['labels'].unique()

#c = train_dataset['labels']=='privacy'
#train_dataset.loc[c,:]

lll=[]
person = {"first name":"majid","last name":"sharghi foroushani","pronoun":"he","birth":"10/08/1997","age":"27","sex":"male","skill":"NLP,LLM","country":"Germany","address":"Philip-9","city":"Erlangen","phone number":"+497773154","available from":"6/24/2024","salary":"10000","German language level":"A1","English language level":"C1","visa":"do not need visa","email":"sharghi.majid@gmail.com","password":"123456","find us":"Google","xing":"www.xing.com","job":"I am stduent","university":"FAU","major":"AI","privacy":"privacy","search":"search","submit":"submit","cookie":"cookie","cover letter":"cover letter","linkedin":"www.linkedin.com","github":"www.github.com","login":"login","register":"register","cv":"/pah/to/cv.pdf","twitter":"www.twitter.com"}
for k,v in person.items():
    c =  train_dataset['labels']==k
    #train_dataset.loc[c,'labels']=k
    pass
    
print(train_dataset['labels'].value_counts())


labels
privacy                   63
search                    43
email                     38
cv                        37
submit                    28
first name                25
phone number              25
cover letter              24
linkedin                  23
last name                 21
find us                   15
cancel                    12
address                   11
salary                    10
available from            10
github                     9
question                   8
transcript                 8
recommendation             5
login                      5
birth                      5
password                   4
german language level      4
xing                       4
next                       4
website                    3
company                    3
house number               3
zip                        3
picture                    3
cookie                     3
city                       3
register                   3
eligible                   2
usernam

In [5]:
test_dataset = pd.read_excel('forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
test_dataset['label'].unique()

array(['cv', 'first name', 'last name', 'email', 'phone',
       'available from', 'guthub', 'privacy', 'submit', 'search',
       'country', 'sex', 'zip', 'cover letter', 'linkedin', 'visa',
       'xing', 'newsletter'], dtype=object)

In [6]:
from transformers import Qwen2ForCausalLM
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct'
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         , device_map='auto',cache_dir=cache_dir).to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/accelerate/utils/modeling.py:1384: UserWarning: Current model requires 117441408 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

In [ ]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [ ]:
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj'
                                                                                     #,'k_proj','o_proj'
                                                                                     ]))
model.print_trainable_parameters()
model

trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=896, out_features=128, bias=True)
              (v_proj): lora.Linear(

In [ ]:
"""from sentence_transformers  import SentenceTransformer

embeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir).to('cuda')
embdeding_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)
embeding_model.eval()
embeding_model"""

"from sentence_transformers  import SentenceTransformer\n\nembeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir).to('cuda')\nembdeding_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)\nembeding_model.eval()\nembeding_model"

In [ ]:

c = test_dataset['label']=='phone'
test_dataset[c]

,query,element,label
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone
14,Telephone Number (Mobile/Work)\n,"<input id=""form-field-field_07a6a9e"" name=""for...",phone
22,Phone *\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone
31,Phone\n *\n,"<input aria-required=""true"" autocomplete=""tel""...",phone
36,"Wir haben festgestellt, dass Sie bereits ein K...","<input data-slnm=""applicationform-input_phoneB...",phone
42,E-Mail*\n Bitte E-Mail-Adresse angeben\n,"<input id=""phone"" kameleoonlistener-m6gb=""true...",phone
46,Telefon\n,"<input autocomplete=""tel"" id=""phone"" maxlength...",phone
58,Phone number\n *\n,"<input aria-invalid=""false"" aria-required=""tru...",phone
64,An email is required\n An email is required\n ...,"<input name=""cPhoneNumber"" ng-model=""candidate...",phone
72,handy\n (optional)\n,"<input name=""phone"" placeholder=""+49 123 2222 ...",phone


In [ ]:
x = test_dataset['query'] + test_dataset['element']
x = x.tolist()
x[0]

'Personal information\n CV or resume\n Browse CV to autocomplete your application. You can still fill your profile manually.\n <input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/>'

In [ ]:
s='['
for i in LABEL_INDEX_TO_KEY[0]:
    s+=f'"{i}",'
    pass

s+=']'
s
#str(LABEL_INDEX_TO_KEY[0])

'["first name","last name","email","password","cv","birth date","phone number","house number","resident country","citizen country","transcript","cover letter","picture","address","linkedin","github","xing","twitter","website","zip","city","salary","sex","available from","captcha","submit","cancel","fake","custom question","privacy","recommend","work here before","find us","dropbox","google drive","work hours","workduration","visa","german language level","company","willing to relocate","university enrollment","english language level","login","username","register","cookie","next","search","other","newsletter",]'

In [ ]:
"""def f(x):
    print(x)
    return '{"answer":"'+ x +'"}'

train_dataset['labels'] = train_dataset['labels'].map(f)
train_dataset"""


'def f(x):\n    print(x)\n    return \'{"answer":"\'+ x +\'"}\'\n\ntrain_dataset[\'labels\'] = train_dataset[\'labels\'].map(f)\ntrain_dataset'

In [ ]:
import random
def get_random():
    d = {}
    for k in person.keys():
        d[k]=str(random.randint(1000,10000))
        
    return d

#get_random()
def to_str(p):
    s='{'
    for k,v in p.items():
        s+=f'"{k}":"{v}",'
        pass

    s+='}'
    return s
    
    
def get_chat():

    d = []
    labels = []
    for i,x in train_dataset.iterrows():
        try:
                p = get_random()
                strp = to_str(p)
                rc = random.choice(list(p.keys()))+' '
                d.append([#{"role":"system","content":""}
                { "role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+strp}
                ,{"role":"user", "content":"What is this HTML element about?\n"+rc+x['input_text'] }
                ,{"role":"assistant","content":'{"answer":"'+ p[x['labels']] +'"}'}
                 ] )
                #labels.append('{"answer":"'+ p[x['labels']] +'"}')
        except:
            #print(x['labels'])
            pass
        
    return d#,labels

get_chat()

[[{'role': 'user',
   'content': 'Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n{"first name":"9992","last name":"3575","pronoun":"5758","birth":"1411","age":"9291","sex":"1037","skill":"7131","country":"1739","address":"2058","city":"4059","phone number":"3245","available from":"7668","salary":"9242","German language level":"3502","English language level":"5664","visa":"7280","email":"4454","password":"4925","find us":"5477","xing":"2758","job":"1488","university":"6537","major":"9008","privacy":"9674","search":"3839","submit":"6438","cookie":"9125","cover letter":"3511","linkedin":"9825","github":"4173","login":"4568","register":"9437","cv":"8143","twitter":"7621",}'},
  {'role': 'user',
   'content': 'What is this HTML element about?\naddress personal informationcv or resumebrowse cv to autocomplete your application. you can still fill your profile manually.<input name="cv" accept="application/pdf,.pdf" type="file" tabindex="-1" style="di

In [ ]:
inp  = get_chat()
tokenized_data = tokenizer.apply_chat_template(inp,add_generation_prompt=False ,truncation=True,max_length=512)  # ,return_tensors='pt')




no_label = tokenizer.apply_chat_template([ [#{"role":"system","content":""}
                                                {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+to_str(person)}
                               ,{"role":"user", "content":"What is this HTML element about?\n"+x['element'] }
                               #,{"role":"assistant","content":""}
                               ] for i,x in test_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

print(len(tokenized_data))

print(tokenizer.decode(token_ids=tokenized_data[0]))
print(tokenizer.decode(token_ids=no_label[0]))


414
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}. Use this content in your answer:
{"first name":"7039","last name":"7352","pronoun":"2070","birth":"4923","age":"8685","sex":"1469","skill":"1962","country":"5864","address":"4482","city":"4322","phone number":"7850","available from":"7694","salary":"9665","German language level":"6973","English language level":"7165","visa":"4979","email":"5285","password":"6552","find us":"6502","xing":"5253","job":"7028","university":"1572","major":"4011","privacy":"6881","search":"3139","submit":"2486","cookie":"1421","cover letter":"1015","linkedin":"8063","github":"4255","login":"3900","register":"8689","cv":"5216","twitter":"3080",}<|im_end|>
<|im_start|>user
What is this HTML element about?
visa personal informationcv or resumebrowse cv to autocomplete your application. you can still fill your profile manually.<input name="cv" accept="application/pdf,.pdf" type=

In [ ]:
tokenizer(['<|im_start|>assistant'])

{'input_ids': [[151644, 77091]], 'attention_mask': [[1, 1]]}

In [ ]:
mydc = DataCollatorForLanguageModeling(tokenizer,False)
def mmf(x):
    y = mydc(x)
    b = (y['labels']==77091).int().argmax()-1
    for i in range(0,b):
        y['labels'][0][i]=-100
        pass
    #print( y['labels'] )
    return y
        
mmf(no_label)['labels'].shape


torch.Size([100, 512])

In [ ]:


trainer =  Trainer(model,
        train_dataset=tokenized_data,
        data_collator=mmf,
        #compute_metrics=compute_metric,
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=2,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          logging_steps=300,
          #eval_accumulation_steps =2,
            
        ))

#trainer.train()

[2024-06-29 12:14:58,999] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


Step,Training Loss
300,0.470700
600,0.180800


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=828, training_loss=0.2937534581059995, metrics={'train_runtime': 104.114, 'train_samples_per_second': 7.953, 'train_steps_per_second': 7.953, 'total_flos': 670747930254336.0, 'train_loss': 0.2937534581059995, 'epoch': 2.0})

In [ ]:
def compute_metric():
    model.eval()
    acc=0
    with torch.no_grad():
        for i in range(len(no_label)):
            x = mydc([no_label[i]])['input_ids'].to('cuda')
            #print(x)
            o= model.generate(input_ids=x,max_new_tokens=20)
            decoded = tokenizer.decode(o[0,x.shape[1]:])
            #print(decoded)
            #break
            
            #print(x[0],x[1])
            try:
                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                print(answer,person[test_dataset.loc[i,'label']])
                c = answer==person[test_dataset.loc[i,'label']]
                if c:
                    acc +=1
                
                pass
            except:
                pass
    
    acc/=len(no_label)
    print("acc:",acc)
    
    
    
compute_metric()

NameError: name 'model' is not defined

In [ ]:
model.eval()
i =11
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                             {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":'email'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
#print(d)
with torch.no_grad():
    
    o = model.generate(input_ids = d,max_length = 512,do_sample=True)
    print(tokenizer.decode(o[0]))
    


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}. Use this content in your answer:
["first name","last name","email","password","cv","birth date","phone number","house number","resident country","citizen country","transcript","cover letter","picture","address","linkedin","github","xing","twitter","website","zip","city","salary","sex","available from","captcha","submit","cancel","fake","custom question","privacy","recommend","work here before","find us","dropbox","google drive","work hours","workduration","visa","german language level","company","willing to relocate","university enrollment","english language level","login","username","register","cookie","next","search","other","newsletter",]<|im_end|>
<|im_start|>user
email<|im_end|>
<|im_start|>user: password<|im_end|>


In [ ]:
$$$

LABEL_INDEX_TO_KEY

SyntaxError: invalid syntax (115010059.py, line 1)

In [ ]:

c = train_dataset['labels']=='relocate'
train_dataset.loc[c,'labels'] = 'willing to relocate'
train_dataset[c]


In [ ]:
#train_dataset.to_excel('train_forms.xlsx',index=0)

In [ ]:
$$$
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [ ]:
def batch_tok(example):

    return embdeding_tokenizer(example,padding=True,truncation=True,max_length=512,return_tensors='pt')



LABEL_INDEX_TO_KEY = list(KEYWORDS.keys())

query_token = embdeding_tokenizer(df_forms_1['query'].tolist(),padding=False,truncation=True,max_length=512)
element_token = embdeding_tokenizer(df_forms_1['element'].tolist(),padding=False,truncation=True,max_length=512)
classes_token = embdeding_tokenizer(LABEL_INDEX_TO_KEY,padding=True,truncation=True,max_length=512,return_tensors='pt')


def myheatmap(x):
    map =[]
    for i in range(len(x)):
        sim = torch.cosine_similarity(x,x[i])
        map.append(sim)
    
    
    plt.figure(figsize=(10,10))
    sns.heatmap(map)
    plt.show()
    
    

with torch.no_grad():
    
    output = embeding_model(**classes_token)
    #class_embdeding = torch.mean(output.last_hidden_state,1)
    class_embdeding = mean_pooling(output,classes_token['attention_mask'])
    class_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)
    #sim = torch.cosine_similarity(class_embdeding,class_embdeding[0])
    
    #print(sim.shape)
    pass
    
i = 151
with torch.no_grad():
    x = torch.tensor(element_token['input_ids'][i:i+1])
    #print(x)
    output = embeding_model(input_ids= x)
    #query_embdeding = torch.mean(output.last_hidden_state,1)
    query_embdeding = mean_pooling(output,torch.tensor(element_token['attention_mask'][i:i+1]))
    #query_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)
    
    sim = torch.cosine_similarity(class_embdeding,query_embdeding)
    print(pd.DataFrame({'value':sim,'label':LABEL_INDEX_TO_KEY}).sort_values('value',ascending=False)[:5])
    print(LABEL_INDEX_TO_KEY[ sim.argmax() ], embdeding_tokenizer.decode(token_ids=x[0]))
    print(df_forms_1['element'][i] )
    

#myheatmap(class_embdeding)
